Estrutura Basica
    * plotly - graficos
    * Flask - aplicação

Como funciona
    * Layout 
        * HTML
        * Dash components (botão)
    * callback (logica dos filtros)
    

Primeiro Dash/ trás flexibilidade, porém o power bi é mais intuitivo. 

In [42]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.


from dash import Dash, html, dcc, Input, Output
import plotly.express as px
import pandas as pd
import dash_auth

USUARIOS = {
    "Agatha": "123456",
}

app = Dash(__name__) #criando seu app Dash
auth =dash_auth.BasicAuth(app, USUARIOS) 
#autenticação profunda pelo dash pago
#autenticação profunda pelo flask(video aula)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options

# importar base de dados
df = pd.read_csv("MOCK_DATA2.csv")

# Converta a coluna "Date of Scraping" para o tipo de data
df['Date of Scraping'] = pd.to_datetime(df['Date of Scraping'])
df['YearMonth'] = df['Date of Scraping'].dt.to_period('M')

# Filtre os dados para mostrar apenas um mês específico (por exemplo, janeiro de 2023)
mes_filtrado = df[df['Date of Scraping'].dt.month == 2]  # 1 representa janeiro
year_filtrado = df[df['Date of Scraping'].dt.year == 2023]  # 1 representa janeiro



# criar graficos


fig = px.bar(df, x="Wine Name", y="Price", color="Store Name", barmode="group")
fig.update_layout(plot_bgcolor='#fcedc1', paper_bgcolor='#fcedc1')

fig2= px.line(mes_filtrado, x="Store Name", y="Price", color="Wine Name")
fig2.update_layout(plot_bgcolor='#fcedc1', paper_bgcolor='#fcedc1')

fig3 = px.scatter(df, x="Date of Scraping", y="Price", color='Wine Name', size_max= 50 )
fig3.update_layout(plot_bgcolor='#fcedc1', paper_bgcolor='#fcedc1')

fig4 = px.pie(df, values='Capacity', names='Wine Name', )
fig4.update_layout(plot_bgcolor='#fcedc1', paper_bgcolor='#fcedc1')


#lista das vinhos
list_wines = list(df["Wine Name"].unique())
list_wines.append("Todas")

#lista das lojas 
list_stores = list(df["Store Name"].unique())
list_stores.append("Todas")


#fig.update_traces(marker=dict(color='#fcedc1'))
#design 
# estrutura titulo 
    # h1 - maior titulo 
    # h2 - um pouco menor
    # h3 - menor ainda

app.layout = html.Div(children=[
    html.H1(children='Sogrape Dash', style={"text-align": "center"} ),

    html.Div(children='''
        Dash: Dasboard de Vendas em python.
    '''),

    html.H4(children=' Price Trends:'), 
    html.H3(children='Vendas de Cada Produto por loja', id='subtitle'), 
    
    
    ###Inserindo um botão
    dcc.RadioItems(list_stores, value="Todas", id='selection_store'),
    dcc.Dropdown(list_wines, value="Todas", id='selection_wine'),
    
    #graficos
    dcc.Graph(id='price-trends'),
    dcc.Graph(id='grafic2'),
    dcc.Graph(id='grafic3',figure=fig3),
    dcc.Graph(id='grafic4',figure=fig4),
    
],style={
       'background-color': '#fcedc1',  # Substitua  'blue' ou '#FFA500'
        'height': '100vh',  # Defina a altura do layout
 })
#callbacks(estrutra padrão)

    #decorator do fflask (linha de codigo que atribui funcionalidade)add output e input
@app.callback(
    Output('subtitle', 'children'), #que eu quero que modifique o botão
    Output('price-trends', 'figure'), #que eu quero que modifique o botão
    Output('grafic2', 'figure'), #que eu quero que modifique o botão
    
    Input('selection_wine', 'value'), 
    Input('selection_store', 'value'), #filtro, quem está modificando, info que ta pedindo\
)
def selecionar_store(stores, wine):
    if stores == "Todas" and wine == "Todas":
        texto = "Vendas de Cada Produto por loja"
        fig = px.bar(df, x="Wine Name", y="Price", color="Store Name", barmode="group")
        fig2 = px.line(mes_filtrado, x="Store Name", y="Price", color="Wine Name")
       
    else:
        df_filter = df
        
        if store != "Todas":
            df_filter = df.loc[df['Store Name'] ==stores, :]
            
        if wine != "Todas":
            df_filter = df.loc[df['Wine'] ==wine, :]
            
        #filtrar as lihas da tabela onde a loja é iguala variavel 
        texto = f"Vendas de Cada Produto por loja {stores}"
        fig = px.bar(df_filter, x="Wine Name", y="Price", color="Store Name", barmode="group")
        fig2 = px.line(df_filter, x="Store Name", y="Price", color="Wine Name")

    return texto, fig, fig2




    

# colocando no ar o site 
if __name__ == '__main__':
    app.run_server(debug=True)


Exception: 